In [ ]:
import sys
import os
import pandas as pd
from datetime import datetime, timedelta

# Add the src directory to the system path and import functions
sys.path.append(os.path.abspath('../src'))
from schedule_weather import generate_activity_list, Scheduler
from weather_constraints import get_daylight_windows, get_tide_windows

from plot_weather import plot_weather

In [ ]:
#read in activity data from excel
#update to be relative to your project structure
main_input_folder_path = r"C:\Users\webbb\VolkerWessels UK\Grp_VMS GeoSeacore Collaboration Space - SZC\2 Internal WIP\06. MST's\Weather Windows"
path = os.path.join(main_input_folder_path, "WeatherWindows.xlsx")
act_df = pd.read_excel(path, sheet_name="Activities", skiprows=1)
constraints_df = pd.read_excel(path, sheet_name="Constraints", skiprows=1)

activities = generate_activity_list(act_df, constraints_df)

c:\Users\webbb\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
act_df.head(30)

,ID,Name,Sub Activity,Predecessor ID(s),Group,Duration (hours),Constraint_ID
0,PM01,Marine spread readiness,Mobilisation of Leask marine spread,NaN,Pipe Management,5,CrewTransfer
1,CTV01,Personel transfer prior to pipe pull,CTV transfer of Personnel to JUB,NaN,Crew Transfer Vessel,2,CrewTransfer
2,HDD01,Pilot Punch Out,Punch out of pilot,"PM01, CTV01",Horizontal Directional Drilling,3,HorizontalDirectionalDrilling
3,CTV02,Bathy Survey,Bathymetric survey of pilot to confirm bore lo...,HDD01,Crew Transfer Vessel,3,BathySurvey
4,HDD02,Pull Back Pilot,Pull back of pilot,HDD01,Horizontal Directional Drilling,6,HorizontalDirectionalDrilling
5,PM02,Unmoor Pipe,Unmooring of pipe from Harwich mooring and hoo...,HDD08,Pipe Management,1,PipeManagement_tidal_slackHW
6,JUB01,JUB positioning,Postioning of JUB on site (including transitio...,CTV02,Jack-up barge,10,JUB_repostioning
7,JUB02,JUB preparation,Readying of JUB to receive PBA,JUB01,Jack-up barge,4,Deck
8,HDD03,Switch to reaming head,Switch pilot head with reaming heads,HDD02,Horizontal Directional Drilling,7,HorizontalDirectionalDrilling
9,HDD04,Reaming of short stop,Reaming of Bore to final diameter (short stop ...,HDD03,Horizontal Directional Drilling,12,HorizontalDirectionalDrilling


In [ ]:
# 1. Define target activity and completion date
target_activity = "Pilot Punch Out"
input_completion_date = datetime(2025, 12, 14, 6, 0)

In [ ]:
# 2. Generate daylight and tidal windows around target date ±7 days
window_data_start = input_completion_date - timedelta(days=5)
window_data_end = input_completion_date + timedelta(days=7)

daylight_windows = get_daylight_windows(
    start_date=window_data_start,
    end_date=window_data_end,
    save_csv=None
)

tidal_data_path = os.path.join(main_input_folder_path, "Reference/harwich_tide.csv")
hw_windows, lw_windows, events = get_tide_windows(
    tidal_data_path,
    start_date=window_data_start,
    end_date=window_data_end,
    skiprows=2,
    datetime_col="Date/Time",
    height_col="Height(m)",
    slack_window_after=1.5,
    slack_window_before=1.5,
    plot=False,
    save_csv=None
)

c:\Users\webbb\Documents\python_local\WeatherOpsPlanner\src\weather_constraints.py:56: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[datetime_col] = pd.to_datetime(df[datetime_col], errors='coerce')


Loaded data from 2025-10-01 00:00:00 to 2026-10-31 23:50:00 with 57024 records.
Filtered data from 2025-12-09 06:00:00 to 2025-12-21 06:00:00 with 1729 records.


In [ ]:
# 3. Create Scheduler instance with activities and environmental windows
scheduler = Scheduler(
    activities=activities,
    daylight_windows=daylight_windows,   # List of (start, end) daylight datetime pairs
    hw_tide_windows=hw_windows,           # High/slack tide windows as datetime pairs
    lw_tide_windows=lw_windows            # Low tide windows as datetime pairs
)

In [ ]:
# 4. Run backward scheduling from target completion date for target activity
scheduled_activities = scheduler.schedule_around_target(target_activity, input_completion_date)



In [ ]:
# 5. Export scheduled activities to pandas DataFrame for analysis
schedule_df = scheduler.to_dataframe()

# 6. Calculate total schedule duration: from earliest start to latest end of activities
total_duration = schedule_df['End'].max() - schedule_df['Start'].min()

# 7. Print total schedule duration
print(f"Total duration of the schedule: {total_duration}")

for i, row in schedule_df.iterrows():
     print(row['Name'])
     #print(row["Constraints"])
     print(row['Weather Restrictions'])


# 8. Display the first few scheduled activities in the DataFrame
schedule_df.head()


Total duration of the schedule: 5 days 00:37:00
Marine spread readiness
{'Maximum Wind Speed at 10m (m/s)': 10.0, 'Maximum Significant Wave Height, Hs (m)': 1.2, 'Maximum Tidal Current (knots)': 2.0}
Personel transfer prior to pipe pull
{'Maximum Wind Speed at 10m (m/s)': 10.0, 'Maximum Significant Wave Height, Hs (m)': 1.2, 'Maximum Tidal Current (knots)': 2.0}
Pilot Punch Out
{}
Bathy Survey
{'Maximum Wind Speed at 10m (m/s)': 10.0, 'Maximum Significant Wave Height, Hs (m)': 0.7, 'Maximum Wave Period (s)': 10.0, 'Maximum Tidal Current (knots)': 2.0}
Pull Back Pilot
{}
Unmoor Pipe
{'Maximum Wind Speed at 10m (m/s)': 12.0, 'Maximum Significant Wave Height, Hs (m)': 0.5, 'Maximum Wave Period (s)': 10.0, 'Maximum Tidal Current (knots)': 2.3}
JUB positioning
{'Maximum Wind Speed at 10m (m/s)': 11.0, 'Maximum Significant Wave Height, Hs (m)': 0.9, 'Maximum Wave Period (s)': 10.0, 'Maximum Tidal Current (knots)': 1.5}
JUB preparation
{'Maximum Wind Speed at 10m (m/s)': 20.0}
Switch to reami

,ID,Name,Start,End,Duration,Group,Description,Predecessor IDs,Successor IDs,Constraints,Weather Restrictions,Earliest Start,Latest End,Float (hours)
0,PM01,Marine spread readiness,2025-12-13 10:40:00,2025-12-13 15:40:00,5.0,Pipe Management,Mobilisation of Leask marine spread,,HDD01,"{'daylight_required': True, 'tide_window_requi...","{'Maximum Wind Speed at 10m (m/s)': 10.0, 'Max...",NaT,2025-12-14 06:00:00,NaN
1,CTV01,Personel transfer prior to pipe pull,2025-12-13 13:40:00,2025-12-13 15:40:00,2.0,Crew Transfer Vessel,CTV transfer of Personnel to JUB,,HDD01,"{'daylight_required': True, 'tide_window_requi...","{'Maximum Wind Speed at 10m (m/s)': 10.0, 'Max...",NaT,2025-12-14 06:00:00,NaN
2,HDD01,Pilot Punch Out,2025-12-14 06:00:00,2025-12-14 09:00:00,3.0,Horizontal Directional Drilling,Punch out of pilot,"PM01, CTV01","CTV02, HDD02, D01","{'daylight_required': False, 'tide_window_requ...",{},NaT,2025-12-14 09:00:00,NaN
3,CTV02,Bathy Survey,2025-12-14 12:30:00,2025-12-14 15:30:00,3.0,Crew Transfer Vessel,Bathymetric survey of pilot to confirm bore lo...,HDD01,JUB01,"{'daylight_required': False, 'tide_window_requ...","{'Maximum Wind Speed at 10m (m/s)': 10.0, 'Max...",2025-12-14 09:00:00,2025-12-14 15:30:00,3.5
4,HDD02,Pull Back Pilot,2025-12-14 09:00:00,2025-12-14 15:00:00,6.0,Horizontal Directional Drilling,Pull back of pilot,HDD01,HDD03,"{'daylight_required': False, 'tide_window_requ...",{},2025-12-14 09:00:00,2025-12-14 15:00:00,0.0


In [ ]:
schedule_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ID                    51 non-null     object        
 1   Name                  51 non-null     object        
 2   Start                 51 non-null     datetime64[ns]
 3   End                   51 non-null     datetime64[ns]
 4   Duration              51 non-null     float64       
 5   Group                 51 non-null     object        
 6   Description           51 non-null     object        
 7   Predecessor IDs       51 non-null     object        
 8   Successor IDs         51 non-null     object        
 9   Constraints           51 non-null     object        
 10  Weather Restrictions  51 non-null     object        
 11  Earliest Start        48 non-null     datetime64[ns]
 12  Latest End            51 non-null     datetime64[ns]
 13  Float (hours)         

In [ ]:
# Load the tidal data
file_path = os.path.join(main_input_folder_path,"Reference\harwich_tide.csv")
tide_df = pd.read_csv(file_path, skiprows=3, names=["DateTime", "Height"])

# Convert the 'DateTime' column to datetime format
tide_df['DateTime'] = pd.to_datetime(tide_df['DateTime'], errors='coerce')

# Filter the data to include only rows where the month is December
december_tide_df = tide_df[tide_df['DateTime'].dt.month == 12]

tide_window_df = events


<>:2: SyntaxWarning: invalid escape sequence '\h'
<>:2: SyntaxWarning: invalid escape sequence '\h'
C:\Users\webbb\AppData\Local\Temp\ipykernel_3972\2389328646.py:2: SyntaxWarning: invalid escape sequence '\h'
  file_path = os.path.join(main_input_folder_path,"Reference\harwich_tide.csv")


In [ ]:
fig = plot_weather(schedule_df, scheduler.daylight_windows, december_tide_df, tide_window_df)

In [ ]:
outputpath=os.path.join(main_input_folder_path, "pipe_pull_schedule.html")
fig.write_html(outputpath, include_plotlyjs='cdn')